In [1]:
from datetime import datetime
from math import exp, pow, sqrt
from time import sleep
import pandas as pd
import plotly.graph_objects as go
from tqdm import tqdm

In [2]:
opt300Risk = pd.read_pickle('etf300Risk.pkl')
opt300Price =  pd.read_pickle('etf300Opt.pkl')

In [3]:
opt300Risk.head(n=1)

,datetime,optionCode,multiplier,option_type,maturity,synthetic_futures_price,underlying_price,is_at_the_money,iv,delta,gamma,theta,vega,rho
0,2020-01-02,OP10002117,10000.0,认购,14.0,4.15169,4.149,False,0.316,0.973614,0.193761,-0.263755,0.061058,0.201378


In [4]:
opt300Price.head(n=1)

,datetime,code,optionCode,optionName,close,open,high,low,preclose,settle,volume,amount,openinterest,openinterest_change
0,2020-01-02,SH510300,OP10002117,300ETF购1月3600,0.5612,0.5283,0.589,0.5283,0.509,0.5614,1947.0,11000605.0,1640,None


In [5]:
# 筛选掉当日到期的期权
opt300Risk = opt300Risk[opt300Risk['maturity'] != 0] 
# 筛选平值认购合约
atmCall = opt300Risk[(opt300Risk['option_type'] == '认购') & (opt300Risk['is_at_the_money'] ==True)]
# 筛选近月平值认购合约，'maturity'最小的
atmCallCur = atmCall.groupby('datetime',as_index=False).apply(lambda x: x[x['maturity'] == x['maturity'].min()]).reset_index(drop=True)
# opt300Price中的合约价格合并到atmCallCur中
portfolio = pd.merge(atmCallCur,opt300Price[['datetime','optionCode','close','optionName']],on=['datetime','optionCode'],how='left')
portfolio.rename(columns={'close':'optionPrice','underlying_price':'etf300Price','multiplier':'optMulti'},inplace=True)
portfolio.head(n=1)

,datetime,optionCode,optMulti,option_type,maturity,synthetic_futures_price,etf300Price,is_at_the_money,iv,delta,gamma,theta,vega,rho,optionPrice,optionName
0,2020-01-02,OP10002122,10000.0,认购,14.0,4.15169,4.149,True,0.151,0.659429,2.431571,-0.565209,0.366123,0.152963,0.0936,300ETF购1月4100


In [6]:
# 可视化'datetime','etf300Price'
fig = go.Figure()
fig.add_trace(go.Scatter(x=portfolio['datetime'], y=portfolio['etf300Price'], name="etf300Price"))
fig.update_layout(title_text='etf300Price')
fig.show()

# backtest
策略：\
保持300etf多头delta份，用1份空头近月平值认购期权合约进行delta对冲\
仓位设置：300etf一侧保持delta份多头，期权合约一侧保持1份空头

In [7]:
commission = 1.7 # 期权手续费
slippage = 5 # 滑点个数

## 多头收益
收益计算逻辑：长期持有delta份300etf，因此多头收益等同于300etf的日收益率曲线

In [8]:
portfolio['etfPosition'] = portfolio['delta'].shift(1) # 需要shift(1)一下，因为是尾盘调仓，所以当日的delta中性仓位是按照昨日的收盘价计算的
portfolio['etfReturn'] = portfolio['etf300Price'].diff() * portfolio['etfPosition']
portfolio['etfReturn'] = portfolio['etfReturn'] - (0.0001 * portfolio['etf300Price']+  0.001*slippage ) * portfolio['etfPosition'].diff() # 买入etf300需要手续费
# 计算累加收益
portfolio['cumEtfReturn'] = portfolio['etfReturn'].cumsum()
# 可视化
fig = go.Figure()
fig.add_trace(go.Scatter(x=portfolio['datetime'], y=portfolio['cumEtfReturn'], name="cumEtfReturn"))
fig.update_layout(title_text='cumEtfReturn')
fig.show()

In [9]:
# portfolio['etfReturn']的分布
fig = go.Figure()
fig.add_trace(go.Histogram(x=portfolio['etfReturn'], name="etfReturn"))
fig.update_layout(title_text='etfReturn')
fig.show()

## 空头收益
使用近月平值认购合约对冲，保持1份空头，等同于合约的日收益

In [10]:
# 按照datetime排序
portfolio.sort_values(by='datetime',inplace=True)
portfolio['optionReturn'] = portfolio.groupby('optionCode').apply(lambda x: x['optionPrice'].diff()).reset_index(drop=True)

# 交易成本
portfolio.loc[portfolio['optionReturn'].isnull(),'optionReturn'] =  2 * commission / 10000 + 0.001 * slippage * 2 # 1.7元/张，0.005*2为滑点

portfolio['optionPosition'] =  -1
portfolio['optionReturn'] = portfolio['optionPosition'] * portfolio['optionReturn']

In [11]:
portfolio['optionCumReturn'] = portfolio['optionReturn'].cumsum()
# 可视化
fig = go.Figure()
fig.add_trace(go.Scatter(x=portfolio['datetime'], y=portfolio['optionCumReturn'], name="optionCumReturn"))
fig.update_layout(title_text='optionCumReturn')
fig.show()

In [12]:
# portfolio['optionReturn']的分布
fig = go.Figure()
fig.add_trace(go.Histogram(x=portfolio['optionReturn'], name="optionReturn"))
fig.update_layout(title_text='optionReturn')
fig.show()

## 组合收益

In [13]:
# 计算组合收益
portfolio['portfolioReturn'] = portfolio['optionReturn'] + portfolio['etfReturn']
portfolio['portfolioCumReturn'] = portfolio['optionCumReturn'] + portfolio['cumEtfReturn']
# 可视化
fig = go.Figure()
fig.add_trace(go.Scatter(x=portfolio['datetime'], y=portfolio['portfolioCumReturn'], name="portfolioCumReturn"))
fig.update_layout(title_text='portfolioCumReturn')
fig.show()

In [14]:
# portfolioReturn的分布
fig = go.Figure()
fig.add_trace(go.Histogram(x=portfolio['portfolioReturn'], name="portfolioReturn"))
fig.update_layout(title_text='portfolioReturn')
fig.show()

In [15]:
# 胜率
len(portfolio[portfolio['portfolioReturn'] > 0]) / len(portfolio[portfolio['portfolioReturn'] != 0])

0.5273684210526316

In [16]:
portfolio['etfName'] = '300ETF'

In [17]:
len(portfolio)

950

In [18]:
select = ['datetime',\
          'optionName','optionPosition','optionPrice','optionReturn','optionCumReturn',\
          'etfName','etfPosition','etf300Price','etfReturn','cumEtfReturn'\
            ,'portfolioReturn','portfolioCumReturn']
portfolio[select].to_csv('portfolio.csv',index=False)

## 业绩归因

In [19]:
def decompose(df): # 对每个合约进行收益分解的函数，传入的df为一个合约的时间序列
    # 按照日期时间排序，保证序列是按照时间顺序排列的
    df = df.sort_values(by=['datetime'])
     # 生成时间变化序列
    df['dt'] = -1*df['maturity'].diff()/242
    # 生成期权合约价格变化序列
    df['dv'] = df['optionPrice'].diff()
    # 生成标的价格变化序列
    df['ds'] = df['synthetic_futures_price'].diff() 
    # 生成iv变化序列
    df['dSigma'] = df['iv'].diff()
    # delta，gamma，vega，theta分别后移一天用于计算
    df['delta_shift'] = df['delta'].shift(1)
    df['gamma_shift'] = df['gamma'].shift(1)
    df['vega_shift'] = df['vega'].shift(1)
    df['theta_shift'] = df['theta'].shift(1)
    
    df['delta_avg'] = df['delta_shift']
    df['deltaProfit'] = df['delta_avg'] * df['ds']
    df['gammaProfit'] = 0.5 * df['gamma_shift'] * (df['ds']**2)
    df['vegaProfit'] = df['vega_shift'] * df['dSigma']
    df['thetaProfit'] = df['theta_shift'] * df['dt']

    df['totalProfit'] = df['deltaProfit'] + df['gammaProfit'] + df['vegaProfit'] + df['thetaProfit']
    df['otherProfit'] = df['dv'] - df['totalProfit']
    df['errorRatio(%)'] = (df['dv'] - df['totalProfit'])/df['dv']*100
    return df

In [20]:
OptDecompose = portfolio.groupby('optionCode',group_keys=False).apply(decompose).reset_index(drop=True)
# 为nan的行用0填充
OptDecompose.fillna(0,inplace=True)
# 归一化
OptDecompose['deltaProfit'] = OptDecompose['deltaProfit'] * OptDecompose['optionPosition']
OptDecompose['gammaProfit'] = OptDecompose['gammaProfit'] * OptDecompose['optionPosition']
OptDecompose['vegaProfit'] = OptDecompose['vegaProfit'] * OptDecompose['optionPosition']
OptDecompose['thetaProfit'] = OptDecompose['thetaProfit'] * OptDecompose['optionPosition']
OptDecompose['otherProfit'] = OptDecompose['otherProfit'] * OptDecompose['optionPosition']
OptDecompose['totalProfit'] = OptDecompose['totalProfit'] * OptDecompose['optionPosition']

deltaProfit = OptDecompose['deltaProfit'].sum()
gammaProfit = OptDecompose['gammaProfit'].sum()
vegaProfit = OptDecompose['vegaProfit'].sum()
thetaProfit = OptDecompose['thetaProfit'].sum()

print('deltaProfit: ',deltaProfit)
print('gammaProfit: ',gammaProfit)
print('vegaProfit: ',vegaProfit)
print('thetaProfit: ',thetaProfit)

# 柱状图
fig = go.Figure()
fig.add_trace(go.Bar(x=['delta','gamma','vega','theta'], y=[deltaProfit,gammaProfit,vegaProfit,thetaProfit]))
fig.update_layout(title_text='profit decompose before delta hedge')
fig.show()

deltaProfit:  0.5684230621706732
gammaProfit:  -1.0857425962577212
vegaProfit:  0.6654456735296621
thetaProfit:  4.747910251055868


In [21]:
OptDecompose['deltaProfit'] = OptDecompose['deltaProfit'] + OptDecompose['etfReturn']
deltaProfit = OptDecompose['deltaProfit'].sum()
gammaProfit = OptDecompose['gammaProfit'].sum()
vegaProfit = OptDecompose['vegaProfit'].sum()
thetaProfit = OptDecompose['thetaProfit'].sum()

print('deltaProfit: ',deltaProfit)
print('gammaProfit: ',gammaProfit)
print('vegaProfit: ',vegaProfit)
print('thetaProfit: ',thetaProfit)

# 柱状图
fig = go.Figure()
fig.add_trace(go.Bar(x=['delta','gamma','vega','theta'], y=[deltaProfit,gammaProfit,vegaProfit,thetaProfit]))
fig.update_layout(title_text='profit decompose after delta hedge')
fig.show()

deltaProfit:  -0.043065488855434764
gammaProfit:  -1.0857425962577212
vegaProfit:  0.6654456735296621
thetaProfit:  4.747910251055868
